<a href="https://colab.research.google.com/github/Ishara20/Text_analysis_using_Python/blob/main/Preparing_Data_For_Predictive_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
import os
#Read course description file
base_file = open("Course-Descriptions.txt", 'rt')
raw_text = base_file.read()
base_file.close()
print("Text read from file : ",raw_text[:200])

#tokenization
import nltk
token_list = nltk.word_tokenize(raw_text)

#Replace special characters
token_list2 = [word.replace("'", "") for word in token_list ]

#Remove punctuations
token_list3 = list(filter(lambda token: nltk.tokenize.punkt.PunktToken(token).is_non_punct, token_list2))

#Convert to lower case
token_list4=[word.lower() for word in token_list3 ]

print("\nSample token list : ", token_list4[:10])
print("\nTotal Tokens : ",len(token_list4))

Text read from file :  In this practical, hands-on course, learn how to do data preparation, data munging, data visualization, and predictive analytics. 
PHP is the most popular server-side language used to build dynamic we

Sample token list :  ['in', 'this', 'practical', 'hands-on', 'course', 'learn', 'how', 'to', 'do', 'data']

Total Tokens :  576


# Building the ngrams DB

In [6]:
from nltk.util import ngrams

#Use a sqlite database to store ngrams information
import sqlite3
conn = sqlite3.connect(":memory:")

#table to store first word, second word and count of occurance
conn.execute('''DROP TABLE IF EXISTS NGRAMS''')
conn.execute('''CREATE TABLE NGRAMS
         (FIRST   TEXT  NOT NULL,
          SECOND  TEXT  NOT NULL,
          COUNTS  INT   NOT NULL,
         CONSTRAINT PK_GRAMS PRIMARY KEY (FIRST,SECOND));''')

#Generate bigrams
bigrams = ngrams(token_list4,2)

#Store bigrams in DB
for i in bigrams:
    insert_str="INSERT INTO NGRAMS (FIRST,SECOND,COUNTS) \
          VALUES ('" + i[0] + "','" + i[1] + "',1 ) \
          ON CONFLICT(FIRST,SECOND) DO UPDATE SET COUNTS=COUNTS + 1"
    conn.execute(insert_str);

#Look at sample data from the table
cursor = conn.execute("SELECT FIRST, SECOND, COUNTS from NGRAMS LIMIT 5")
for gram_row in cursor:
    print("FIRST=", gram_row[0], "SECOND=",gram_row[1],"COUNT=",gram_row[2])


FIRST= in SECOND= this COUNT= 2
FIRST= this SECOND= practical COUNT= 1
FIRST= practical SECOND= hands-on COUNT= 1
FIRST= hands-on SECOND= course COUNT= 1
FIRST= course SECOND= learn COUNT= 1


#Recommending next word

In [7]:
#Function to query DB and find next word
def recommend(str):
    nextwords = []
    #Find next words, sort them by most occurance
    cur_filter = conn.execute("SELECT SECOND from NGRAMS \
                              WHERE FIRST='" + str + "' \
                              ORDER BY COUNTS DESC")

    #Build a list ordered from most frequent to least frequent next word
    for filt_row in cur_filter:
        nextwords.append(filt_row[0])
    return nextwords

#Recommend for words data and science
print("Next word for data ", recommend("data"))
print("\nNext word for science ", recommend("science"))

Next word for data  ['science', 'analysis', 'data', 'from', 'in', 'mining', 'munging', 'node.js', 'preparation', 'scientists', 'visualization', 'you']

Next word for science  ['begins', 'requires', 'specialists', 'teams']
